In [56]:
import pandas as pd
import requests
import numpy as np
from requests_html import HTML,HTMLSession
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import random
from random import choice,randrange
from sklearn.neighbors import NearestNeighbors
import math
import time
from torch_geometric.nn import GCNConv,GraphConv,GATConv,SAGEConv

## Loading of the dataset Olga

 - This is the dataset provided by the paper's author **'Artist similarity with Graph Neural Networks'**.
 - Along the columns we have information about the Musicbrainz_id of an artist, its partition in the dataset (train,val,test), and the AcousticBrainz low level features of the artist, taken from a sample of 25 songs. 



In [3]:
olga=pd.read_csv('olga.csv')
#train 0-14138, #val 14139-15905, #test 15906-17673 (indices)
olga.head()  

,index,musicbrainz_id,partition,tracks
0,0,c5b11a19-5ba6-4554-a65b-e505c3296d48,train,"['0376162c-24b4-4b52-a351-185e40de9b71', '1607..."
1,1,252ca659-19c6-46e1-a464-c4b80300bb02,train,"['029aebb5-4991-435a-90db-f56122c8fc4c', '0432..."
2,2,b9f29919-5d08-498d-bcfc-fda33deceade,train,"['073902fd-e188-416d-aeec-49c9c7fdf4d7', '0984..."
3,3,1b396d40-a35a-4558-b292-0b2685f7ea8f,train,"['0a3e1513-cc41-4234-9245-3f15adefef46', '1b07..."
4,4,84cff711-6d4f-49b9-bc54-f4db4c7addfb,train,"['02911df4-a535-4da0-bd0f-06027f31da6c', '091b..."


### How do we retrieve the Graph topology?

- Thanks to the musicbrainz_id of each artist, we can get the link to its AllMusic profile, and from there we get also the information about its related artists. Each link is related to a unique artist, indeed we can spot a 12 numbers identifier for each of these.
- After having obtained the AllMusic link for each artist in the dataset (if exists), we want to associate to each artist its related ones. We do this just for those that can be re-mapped in the dataset's musicbrainz_ids, because we have associated tracks features for those.

In [4]:
class DatasetOlga(): #In this class, we obtain through different methods the main characteristics of the graph of artists
                    # thanks to the available information in the olga dataset
    def __init__(self,olga):
        self.olga=olga
        self.mb=olga.musicbrainz_id
        self.artists={} #Needed for obtaining the mapping from musicbrainz to the allmusic ids
        self.l=len(self.mb)
        self.d={}       #Needed for obtaining a dict. where keys are artists, and values are the artists similar to them, based on self.artists
        self.NI={}      #Dict. that will contain the artist's features
    
    def get_mapping(self,i):                                                                                                 #This method returns the allmusic page of an artist (if exists), given his id from the dataset 
        response = requests.get(f'https://musicbrainz.org/ws/2/artist/{str(self.mb[i])}?inc=url-rels&fmt=json')
        if response.ok:
            data = response.json()
            refs = [r['url']['resource'] for r in data['relations'] if r['type'] == 'allmusic']        
            return refs[0] if len(refs) != 0 else "Not found"

        

    def get_mappingList(self,init,end,increm=500):
        Lmusicbrainz_id=self.mb[init:end] #We can specify the range of the artists of our interest, for the purpose of this NN task
        length=len(Lmusicbrainz_id)       #we will take all of them into consideration.
        c=0
        for i in range(len(Lmusicbrainz_id)):
            mapp=self.get_mapping(i)   #get_mapping method again.
            if mapp==None:
                while mapp==None:
                    mapp=self.get_mapping(i)
                    
            if mapp!="Not found":   #Some of the ids has not a respective allmusic id, so we lose that information
                mapp=str(mapp)      #Mapp are strings of links
                key=mapp[-12:]
                self.artists[key]=i
            c+=1
            if c%increm==0 or c==30:
                    print("{}/{} artists were processed".format(c,length)) #This is just to keep track of the processed artist
                    
            
        self.save_data(self.artists,'MsbMapped1.json')  #We do save the Artists Ids map, this function, when called, takes a lot
                                                        #of time, for this reason its result is already saved in the file:
        return self.artists                             # 'MsbMapped1.json'
    
    
    def get_GraphDict(self,name='MsbMapped1.json',increm=500):
        session=HTMLSession()
        c=0 #Counter
        artID=self.load_data(name) #We load the mapped artists (between MusicBrainz Ids, and AllMusic Ids)
        length=len(artID.keys())
        for k in artID.keys(): #dict of mapped mbids, this has to be computed before from getmapping
            if k!=None:
                url='https://www.allmusic.com/artist/'+ k+ '/related' #k is just the code, every link for the artist is distinguished 
                r=session.get(url)                                    #by a unique code in the link.
                sess=r.html.find('body',first=True)
                div=sess.find('.overflow-container')                  #The information of the related artists are exctracted
                divn=div[0]                                           #from the html of the allmusic's related web page
                divn=divn.find('.content-container')
                divn=divn[0]
                divn=divn.find('.content')
                divn=divn[0]
                divn=divn.find('section',first=True)
                if divn==None:
                    self.d[artID[k]]=[] #That artist has not related artists (or we have missing information)
                    continue
                artists=divn.find('li')
                artistL=[]


                for i in range(len(artists)):
                    art=artists[i]
                    art=art.find('a')            #We look for all the k's related artists links
                    link=list(art[0].absolute_links)[0] #Absolute_link returns a one-element set, that we convert into a list and
                    link=str(link)[-12:]                #we get its code
                    if link in artID.keys(): #g is the dict of all the mapped musicbrainz_ids
                        artistL.append(self.artists[link]) #Some of the related artists may not be in the musicbrainz_ids list.
                self.d[artID[k]]=artistL
                c+=1
                if c%increm==0 or c==30:
                    print("{}/{} artists were processed".format(c,length))
        self.save_data(self.d,'graphSimilarities1.json') #Here we save the connection amongst the artists, obtained with this method
        print("Done...")     #Also it takes some time to process, for this reason the result of this method can be 
        return self.d        #found at the 'graphSimilarities.json' file.
    
    def save_data(self,dicti,name):
        jfile = open(name, "w")
        jfile = json.dump(dicti, jfile)
    
    def load_data(self,name):
        jfile = open(name, "r")
        dicti = json.load(jfile)
        return dicti


## Graph construction

- Once we have obtained the information necessary to construct the Graph topology, and stored them into two json files ('MsbMapped.json','graphSimilarities.json'), we still need to have the Graph data structure needed to feed a Graph Convolutional Network.

- We then define the adjacency matrix of the graph from the 'graphSimilarities.json' previously obtained, and the features of each artist (or graph's node) are obtained from a numpy array stored in the file 'acousticbrainz.npy', such file was provided by the paper's authors.

In [5]:
n_features=2613
class Graph():  #The purpose of this class is to construct the graph of artists, in particular the Adjacency matrix A, and the  
                # node features tensor X
        
    def __init__(self,mapfile,gfile):  #The expected files are the ones mentioned before.
        self.mfile=self.load_data(mapfile)
        self.gfile=self.load_data(gfile)
        self.A=torch.zeros((len(self.mfile),len(self.mfile)))
        self.X=torch.zeros((n_features,len(self.mfile)))
        self.ord=sorted(list(map(int,self.gfile.keys())))
        self.enc1={}
        self.enc2={}
    
    #With the preprocessing step at the previous cell we have lost some information
    #and also the ordering of the artists, so i have defined a method that for each previous artist index
    #we can encode it to a new ordered list of artists.
    
    
    def encoding1(self):   #From ordered to unordered, Dict are not ordered data structures, so is better to order them before
        for k in range(len(self.mfile)): #This encoding is used to get the Instance matrix
            self.enc1[k]=self.ord[k]
        return self.enc1
    
    def encoding2(self):   #From unordered to ordered,  From real number, to ordered one.
        for k in range(len(self.mfile)): #This encoding is used to get the Adjacency matrix
            self.enc2[self.ord[k]]=k
        return self.enc2
    
    def get_instance(self,instances,df=False):#We take the features centroid, obtained from 25 track from artists discographies.
        X=np.load(instances)                  #The instances file is provided by the repository mentioned in the paper.
        X=torch.from_numpy(X).requires_grad_(True) #We take the allmusicIDs, which contain the key of the artists for which we haven't 
        c=0                                        # lost information
        enc=self.encoding1()
        for k in self.mfile:
            z=enc[c]
            self.X[:,c]=X[z] 
            c+=1
        return self.X
    
    def get_adjacency(self,symmetry=False,df=False):  #The hypothesis could be either a symmetric matrix (paper), or not.
        enc=self.encoding2()
        for k in self.gfile:
            c1=enc[int(k)]
            for j in self.gfile[k]:
                c2=enc[int(j)]
                if self.A[c2,c1]==1 and symmetry==True:
                    continue
                self.A[c1,c2]=1
                if symmetry:
                    self.A[c2,c1]=1

            
        return self.A
    
    
    def load_data(self,name):
        jfile = open(name, "r")
        dicti = json.load(jfile)
        return dicti
    
g=Graph('MsbMapped.json','graphSimilarities.json')
X1=g.get_instance('acousticbrainz.npy')
A1=g.get_adjacency(symmetry=True)


# GraphSAGE model

- Once we also have the features for each instance, and the adjacency matrix of the graph, we can start to design the Graph Convolutional Layers, and the Fully Connected layers, as described in the paper.

- Every feature vector has 2613 elements (low level features of the artist), our aim is to embed these vectors in a 100-dimensional space, and thanks to the Neural Networks we want to described it as a space where the similarity of each artist is described by the Euclidean distance.

- The GraphNN that the paper's authors decided to use is the GraphSAGE model(SAGE stands for Sample and AGgregatE). 

- This approach propose a framework that generalizes the GCN to use trainable aggregation functions (beyond simple convolutions). 

In [51]:
#Here we have some hyperparamters, and also the list of ordered artists, and their indices with respect their own set (train,val,test).
outf=[2613,256,256,256,100] #Train: 0:9021, #Val: 9022:10189, #Test: 10190:11260
train_=list(range(0,9021+1)) #Train set, without val
train=list(range(0,10189+1)) #Train set, with val
val=list(range(9022,10189+1))
test=list(range(10190,11260+1))
KNN=200       #K-nearest-neighbors for the evaluation metrics.
class GraphSAGE(nn.Module):
    
    def __init__(self,X,A,train_set,L,batch_size,device):
        super(GraphSAGE,self).__init__()
        self.device=device
        self.A=A.to(self.device) #Tensors version of adjacency matrix and Instances
        self.X=torch.tensor(X.tolist(),requires_grad=True).to(self.device)   
        self.COO=torch.load('COOA.pt')
        self.train=train_set
        self.L=L
        self.lamb=0.8
        self.bs=batch_size
        self.mb=self.mini_batches(self.train,self.bs)
        self.diz=self.getDiz()
        self.OrDiz=self.getCorrenspondancies(self.mb)
        self.SG1=SAGEConv(2613,256,normalize=True,aggr="mean")
        self.SG2=SAGEConv(256,256,normalize=True,aggr="mean")
        self.SG3=SAGEConv(256,256,normalize=True,aggr="mean")
        
        self.FC1=nn.Linear(256,256)
        self.FC2=nn.Linear(256,256)
        self.FC4=nn.Linear(256,256)
        
        self.FC3=nn.Linear(256,100)
        
        
    def forward(self,V,nbs=-1):
        Vdiz=self.tracing(V)
        if len(V)>500 and nbs!=-1:
            OrDiz=self.OrDiz[nbs]
        else:
            OrDiz=self.getCorr(V)
            
        for k in range(0,self.L):  
            

            if k==0:
                Es=self.select(self.X,set(),Vdiz[k+1]).T
                Anew=self.select(self.A,Vdiz[k+1],Vdiz[k+1])
                Anew=self.ConvertAtoCOO(Anew)
                Es=self.SG1(Es,Anew).T
    
                
            if k==1:
                Es=self.select(Es,set(),OrDiz[k+1].keys()).T
                Anew=self.select(self.A,Vdiz[k+1],Vdiz[k+1])
                Anew=self.ConvertAtoCOO(Anew)
                Es=self.SG2(Es,Anew).T
            
            if k==2:
                Es=self.select(Es,set(),OrDiz[k+1].keys()).T
                Anew=self.select(self.A,Vdiz[k+1],Vdiz[k+1])
                Anew=self.ConvertAtoCOO(Anew)
                Es=self.SG3(Es,Anew).T
            
            if k==self.L-1:
                Es=self.select(Es,set(),OrDiz[k+2]).T
            
            
        out=self.FC1(Es)
        out=self.FC2(out)
        out=self.FC3(out)
        return out.T
    
    def getCorrenspondancies(self,mbb):
        OrDiz={}
        for j in range(len(mbb)):
            Vdiz=self.tracing(mbb[j])
            OrDiz[j]={}
            for k in Vdiz:
                OrDiz[j][k]={}
                OrDiz[j][k]={i: sorted(list(Vdiz[k]))[i] for i in range(len(Vdiz[k]))}
            print("Processed {}-th mini-batch".format(j+1))
        return OrDiz
    def getCorr(self,mbb):
        OrDiz={}
        Vdiz=self.tracing(mbb)
        for k in Vdiz:
            OrDiz[k]={}
            OrDiz[k]={i: sorted(list(Vdiz[k]))[i] for i in range(len(Vdiz[k]))}
        return OrDiz
            
            
    
    def tracing(self,V):
        Vdiz={}
        K=self.L+1
        Vdiz[K]=sorted(list(V))
        for k in range(K-1,0,-1):
            d=set()
            for idx in Vdiz[k+1]: 
                d=d.union(self.get_n(idx))
                
            Vdiz[k]=d
        return Vdiz
            
    def get_n(self,idx): #This function is the neighbor's function. Given a batch index we get its neighborhood.
        t=torch.nonzero(self.A[idx])
        s=set()
        
        for k in t:
            if t.shape[0]!=0:
                s.add(k.item())
        s.add(idx)     
                
        return s
    def select(self,mat,row,col):  #Given a set of indices for rows or column or both, we get the respective elements.
        col=sorted(list(col))      #This is applied when we get the t matrix.
       
        c=0
        if row==set():
            col=torch.tensor(col,dtype=torch.int32).to(self.device)
            ma=torch.index_select(mat,1,col)
            return ma
        else:
            row=torch.tensor(sorted(list(row))).to(self.device)
            col=torch.tensor(col).to(self.device)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    
    def tfunc(self,n_feat,es,V):               #This is the t function, which was previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    def tfunc2(self,n_feat,es,V,prev):               #This tfunction is later used for the accuracy evaluation step
        V=sorted(list(V))
        c=0
        for k in V:
            prev[:,k]=es[:,c]
            c+=1
        return prev
    
    def getDiz(self):
        diz={}
        for k in range(self.COO.shape[1]):
            cn=int(self.COO[0][k].item())
            if cn>self.train[-1]:
                break
            
            if cn not in diz:
                diz[cn]=[[int(self.COO[1][k])]]
                r=randrange(len(self.train))
                while self.A[int(cn)][r]!=0:
                    r=randrange(len(self.train))
                diz[cn].append([r])
            elif cn in diz:
                diz[cn][0].append(int(self.COO[1][k]))
                r=randrange(len(self.train))
                while self.A[int(cn)][r]!=0:
                    r=randrange(len(self.train))
                diz[cn][1].append(r)
                
        return diz
    def ConvertAtoCOO(self,SA):
        Anew=torch.nonzero(SA).T.type(torch.LongTensor)
        return Anew.to(self.device)

    def getHardN(self,a,t):
        NEG=list(set(self.diz[a][1].copy()).intersection(set(self.train)))
        
        if len(NEG)==0:
            return "no"
        
        elif len(NEG)==1:
            HN=self.forward([int(NEG[0])])
            HN=torch.reshape(HN,(1,100))
            return HN #,int(NEG[0])]
        
        
        l=[]
        c=0
        while len(l)<4 and len(NEG)>0:
            s=randrange(1000)
            r=randrange(len(NEG))
            random.seed(s)
            NEG1=NEG
            if NEG[r] in self.train and NEG[r] not in l:
                l.append(int(NEG.pop(r)))
                c=0
            elif NEG1==NEG:
                c+=1
                if c==20:
                    break
            else:
                continue
        HNl=self.forward(l)
        HNd1={}
        HNd2={}
        summ=0
        if len(l)==1:
            HN=self.forward([l[0]])
            HN=torch.reshape(HN,(1,100))
            return HN #,int(l[0])]
        elif len(l)==0:
            return "no"
        
        for k in l:
            H1=HNl[:,l.index(k)]
            HN=torch.reshape(H1,(100,1))
            
            dis=((t-HN)**2).sum().item()
            HNd1[k]=H1
            HNd2[k]=dis
            summ+=dis
        keys=list(HNd1.keys())
        
        for k in keys:
            HNd2[k]/=summ
            if HNd2[k]<=1-self.lamb:
                HNd2.pop(k)
                HNd1.pop(k)
        key=min(HNd2,key=HNd2.get)
        res=torch.reshape(HNd1[key],(1,100))
        return res   #,key]    
        
    def getHardP(self,a,t):
        POS=list(set(self.diz[a][0].copy()).intersection(set(self.train)))
        
        if len(POS)==0:
            return "no"
        
        elif len(POS)==1:
            HP=self.forward([int(POS[0])])
            HP=torch.reshape(HP,(1,100))
            return HP #,int(POS[0])]
        
        
        l=[]
        c=0
        while len(l)<4 and len(POS)>0:
            s=randrange(1000)
            r=randrange(len(POS))
            random.seed(s)
            POS1=POS
            if POS[r] in self.train and POS[r] not in l:
                l.append(int(POS.pop(r)))
                c=0
            elif POS1==POS:
                c+=1
                if c==20:
                    break
            else:
                continue
        HPl=self.forward(l)
        HPd1={}
        HPd2={}
        summ=0
        if len(l)==1:
            HP=self.forward([l[0]])
            HP=torch.reshape(HP,(1,100))
            return HP  #,int(l[0])]
        elif len(l)==0:
            return "no"
        for k in l:
            H1=HPl[:,l.index(k)]
            HP=torch.reshape(H1,(100,1))
            
            dis=((t-HP)**2).sum().item()
            HPd1[k]=H1
            HPd2[k]=dis
            summ+=dis
        keys=list(HPd1.keys())
        for k in keys:
            HPd2[k]/=summ
            if HPd2[k]>=self.lamb:
                HPd2.pop(k)
                HPd1.pop(k)
        key=max(HPd2,key=HPd2.get)
        res=torch.reshape(HPd1[key],(1,100))
        
        return res   #,key]
        
        
    
    
    def mini_batches(self,indices,bs=32): #This function generates a list of minibatches, of size bs
        indicesN=indices.copy()           #sets are unordered data structure, so there is no need to shuffle them. 
        mbList=[]                         #Lists of lists of mini_batches indices 
        while len(indicesN)!=0:
            mb=set()                      #Inner list, with the indices of a particular mini_batch
            while len(mb)<bs:
                if len(indicesN)==0:
                    mbList.append(mb)
                    return mbList
                r=choice(indicesN)
                sample=indicesN.pop(indicesN.index(r))
                mb.add(sample)
            mbList.append(mb)
        return mbList          #obj.mini_batches(#,bs=128) #: train_,train,val,test, we get lists of list of batches from here
    
    
    def calcG(self,ID):  #This method is used for the evaluation of accuracy, in particular it computes the denominator
        if ID>200:       # as described in the paper.
            ID=200
        c=1
        somm=0
        while c<=ID:
            somm+=1/(math.log2(1+c))
            c+=1
        return somm

    def evalAcc(self,S,kneigh,test_set):  #This function is to compute accuracy for the test and train set. 
        S=sorted(S)
        T=self.forward(S)
        T=T.detach().numpy().transpose()
        neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)#With the K-NN we get the nearest 
        dist,ind=neigh.kneighbors(T)   
        acc=[]
        A_acc=self.A[0:S[-1]+1,0:S[-1]+1]
        for k in test_set:
            summ=0
            ideal=A_acc[k,:].sum().item()  #gs
            den=self.calcG(ideal)
            if den==0:#There is the problem of the distance for the people without neighbors
                continue  #1-(n/200) or ignore them.
            for j in range(len(ind[k][1:])):
                if A_acc[k][ind[k][1:][j]]!=0:
                    summ+= 1/(math.log2(1+(j+1)))
                    
                else:
                    continue
                
            summ/=den
            acc.append(summ)
        return acc
    




### Training step

- In the following cell is possible to choose different hyperparameters to train the network.
- In the hyperparameters tuning we must take into account: the number of layer (they try from 0 to 3 graph layers), the batch size, and the dimension of the projection matrices (in the aggregation step).
- There are also other hyperparameters of course, but  we have the description in the paper for them.

In [60]:
training=train_
testing=val
n_layer=1#n_of graph conv.layer.
batch_size=512 #This is the batch size used in the paper which inspired artist similarity
device=torch.device("cpu")
gs=GraphSAGE(X1,A1,training,n_layer,batch_size,device).to(device)
num_epochs=50 #According to the paper there will be 50 epochs for each experiment 
out_feat=100
triplet_loss = nn.TripletMarginLoss(margin=0.2, p=2)
lr=min([1,(1-0.9)/2])    #linear warm-up described in the paper, beta2 = 0.9
optimizer=torch.optim.Adam(gs.parameters(),lr=lr,weight_decay=0)
scheduler=lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min= 0, last_epoch= -1, verbose=True)

Processed 1-th mini-batch
Processed 2-th mini-batch
Processed 3-th mini-batch
Processed 4-th mini-batch
Processed 5-th mini-batch
Processed 6-th mini-batch
Processed 7-th mini-batch
Processed 8-th mini-batch
Processed 9-th mini-batch
Processed 10-th mini-batch
Processed 11-th mini-batch
Processed 12-th mini-batch
Processed 13-th mini-batch
Processed 14-th mini-batch
Processed 15-th mini-batch
Processed 16-th mini-batch
Processed 17-th mini-batch
Processed 18-th mini-batch
Adjusting learning rate of group 0 to 5.0000e-02.


In [ ]:
#With these lines of code we obtain the embedded space sample
start=time.time()
history_loss={}
history_accuracy={}
mbb=gs.mb
for epoch in range(num_epochs):
    print("Processing epoch n°",epoch+1)
    num=int(len(training)/batch_size)+1
    startEP=time.time()
    history_loss[epoch+1]=[]
    for k in range(len(mbb)):
        print("Processing {}-th epoch: {}/{} mini-batch".format(epoch+1,k+1,num))
        startMB=time.time()
        
        Ex=gs(mbb[k],nbs=k)
        anchors=Ex.T
        mbbord=sorted(list(mbb[k]))

        for j in range(len(mbbord)):
            #print(j)
            if mbbord[j] in gs.diz and j!=0: #If the sample is not the first of the training pipeline
                neg=torch.reshape(anchors[j],(100,1))
                neg2=gs.getHardN(mbbord[j],neg)
                if neg2!="no":  #If the sample has negatives samples
                    negatives=torch.cat((negatives,torch.reshape(neg2,(1,100))))
                   
                    pos=torch.reshape(anchors[j],(100,1))
                    pos2=gs.getHardP(mbbord[j],pos)
                    if pos2!="no":  #If the sample has positives samples
                        positives=torch.cat((positives,pos2))
                        
                        continue
                    else: #If the sample has NOT positives samples
                        pos=torch.reshape(anchors[j],(1,100))
                        positives=torch.cat((positives,pos))
                        
                        continue

                else:  #If the sample has NOT negatives samples
                    neg=torch.reshape(neg,(1,100))
                    negatives=torch.cat((negatives,neg))
                    
                    pos=torch.reshape(anchors[j],(100,1))
                    pos2=gs.getHardP(mbbord[j],pos)
                    if pos2!="no": #If the sample has NOT negatives samples, but has positives samples
                        positives=torch.cat((positives,pos2))   
                        continue
                    else:
                        pos=torch.reshape(anchors[j],(1,100))
                        positives=torch.cat((positives,pos))
                        
                        continue



            elif j==0 and mbbord[j] in gs.diz:
                neg=gs.getHardN(mbbord[j],torch.reshape(anchors[j],(100,1)))
                pos=gs.getHardP(mbbord[j],torch.reshape(anchors[j],(100,1)))
                if neg!="no":
                    negatives=neg
                    
                    if pos!="no":
                        positives=pos 
                        
                        continue
                    else:
                        positives=torch.reshape(anchors[j],(1,100))
                        
                        continue

                else:
                    negatives=torch.reshape(anchors[j],(1,100))
                    
                    if pos!="no":
                        positives=pos 
                        
                        continue
                    else:
                        positives=torch.reshape(anchors[j],(1,100))
                        
                        continue

            elif mbbord[j] not in gs.diz and j!=0:
                negatives=torch.cat((negatives,torch.reshape(anchors[j],(1,100))))
               
                pos=torch.reshape(anchors[j],(1,100))
                positives=torch.cat((positives,pos))
                
                continue

            elif mbbord[j] not in gs.diz and j==0:
                negatives=torch.reshape(anchors[j],(1,100))
                positives=torch.reshape(anchors[j],(1,100))

                continue
                

           

   
        endD=time.time()       
        
        optimizer.zero_grad()
        loss=triplet_loss(anchors,positives.detach().to(device),negatives.detach().to(device))
        loss.backward()
        optimizer.step()
        history_loss[epoch+1].append(loss.item())
        
        
    with torch.no_grad():
        history_loss[epoch+1]=sum(history_loss[epoch+1])/len(history_loss[epoch+1])
        print("Evaluating the epoch n°",epoch+1)
        evalSet=training + testing
        acc=gs.evalAcc(evalSet,KNN,testing)
        TestAcc=sum(acc)/len(acc)
        history_accuracy[epoch+1]=TestAcc
        print("Processesed epoch n° {},\tTest accuracy: {:.4f}\tLoss: {:.8f}".format((epoch+1),TestAcc,history_loss[epoch+1]))
        endEP=time.time()
        scheduler.step()
        print("Requested time for processing {}-th epoch was: {:.4f} secs.".format(epoch+1,endEP-startEP))


Processing epoch n° 1
Processing 1-th epoch: 1/18 mini-batch
Processing 1-th epoch: 2/18 mini-batch
Processing 1-th epoch: 3/18 mini-batch
Processing 1-th epoch: 4/18 mini-batch
Processing 1-th epoch: 5/18 mini-batch
Processing 1-th epoch: 6/18 mini-batch
Processing 1-th epoch: 7/18 mini-batch
Processing 1-th epoch: 8/18 mini-batch
Processing 1-th epoch: 9/18 mini-batch
Processing 1-th epoch: 10/18 mini-batch
Processing 1-th epoch: 11/18 mini-batch
Processing 1-th epoch: 12/18 mini-batch
Processing 1-th epoch: 13/18 mini-batch
Processing 1-th epoch: 14/18 mini-batch
Processing 1-th epoch: 15/18 mini-batch
Processing 1-th epoch: 16/18 mini-batch
Processing 1-th epoch: 17/18 mini-batch
Processing 1-th epoch: 18/18 mini-batch
Evaluating the epoch n° 1
Processesed epoch n° 1,	Test accuracy: 0.0661	Loss: 42.18864505
Adjusting learning rate of group 0 to 4.9951e-02.
Requested time for processing 1-th epoch was: 271.0695 secs.
Processing epoch n° 2
Processing 2-th epoch: 1/18 mini-batch
Proce

In [60]:
# positives=torch.randn(512,100)
# negatives=torch.randn(512,100)
# anchors=d.T
# loss=triplet_loss(anchors,positives.detach(),negatives.detach())
# loss.backward()
print(loss)

tensor(0.5046, grad_fn=<MeanBackward0>)


In [11]:
n_feat=2613
class Provanet(nn.Module):
    
    def __init__(self,X,A,L):
        super(Provanet,self).__init__()
        self.X=torch.tensor(X.tolist(),requires_grad=True)
        self.A=A
        self.Anew=torch.load('COOA.pt')
        self.L=L
        self.SG1=SAGEConv(2613,256,normalize=True,aggr="mean")
        self.SG2=SAGEConv(256,256,normalize=True,aggr="mean")
        self.SG3=SAGEConv(256,256,normalize=True,aggr="mean")
        
        self.FC1=nn.Linear(256,256)
        self.FC2=nn.Linear(256,256)
        self.FC4=nn.Linear(256,256)
        
        self.FC3=nn.Linear(256,100)
        
        
    def forward(self,V):
        Vdiz=self.tracing(V)
        
        for k in range(0,self.L):  
            
                
            if k==0:
                Es=self.select(self.X,set(),Vdiz[k+1]).T
                Anew=self.select(self.A,Vdiz[k+1],Vdiz[k+1])
                Anew=self.ConvertAtoCOO(Anew)
                Es=self.SG1(Es,Anew).T
                
            if k==1:
                Es=self.select1(Es,set(),Vdiz[k+1]).T
                Anew=self.select(self.A,Vdiz[k+1],Vdiz[k+1])
                Anew=self.ConvertAtoCOO(Anew)
                Es=self.SG2(Es,Anew).T
            
            if k==2:
                Es=self.select1(Es,set(),Vdiz[k+1]).T
                Anew=self.select(self.A,Vdiz[k+1],Vdiz[k+1])
                Anew=self.ConvertAtoCOO(Anew)
                Es=self.SG3(Es,Anew).T
            
            if k==self.L-1:
                Es=self.select1(Es,set(),Vdiz[k+2]).T
            
            
        out=self.FC1(Es)
        out=self.FC2(out)
        out=self.FC3(out)
        return out.T
    
    def select(self,mat,row,col):  
        col=sorted(list(col))      
        if row==set():
            mat=torch.index_select(mat,1,torch.tensor(col))
            return mat
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def select1(self,mat,row,col):  
        col=list(range(0,len(col)))      
        if row==set():
            mat=torch.index_select(mat,1,torch.tensor(col))
            return mat
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def ConvertAtoCOO(self,SA):
        Anew=torch.nonzero(SA).T.type(torch.LongTensor)
        return Anew
    
    def tracing(self,V):
        Vdiz={}
        allN=set()
        K=self.L+1
        Vdiz[K]=sorted(list(V))
        for k in range(K-1,0,-1):
            d=set()
            for idx in Vdiz[k+1]: 
                d=d.union(self.get_n(idx))
                
            Vdiz[k]=d
        return Vdiz
        
    
    def get_n(self,idx):
        t=torch.nonzero(self.A[:len(training),:len(training)][idx])
        s=set()
        
        for k in t:
            if t.shape[0]!=0:
                s.add(k.item())
        s.add(idx)
        
        return s
    
    def tfunc(self,n_feat,es,V):               #This is the t function, which was previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    
    
        
    

RuntimeError: The size of tensor a (30) must match the size of tensor b (126192) at non-singleton dimension 0

In [82]:
torch.nonzero(A1).T

torch.Size([2, 126192])

In [12]:
num_epochs=20
# 
n_layer=1
p=Provanet(X1,A1,n_layer)
P1=torch.randn((512,100))
N1=torch.randn((512,100))
#c1=Conf3(X1,A1)
criterion=nn.TripletMarginLoss(margin=0.2,p=2)
optimizer=torch.optim.Adam(p.parameters(),lr=0.001)
for epoch in range(num_epochs):
    
    y=p([i for i in range(0,512)]).T
#     P1=torch.transpose(p([i for i in range(100,155)]),0,1)
#     N1=torch.transpose(p([i for i in range(200,205)]),0,1)
    start=time.time()
    loss=criterion(y,P1,N1)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())
    end=time.time()
    print(end-start)

    

0.5170881152153015
1.1947298049926758
0.4803261458873749
1.176288366317749
0.4558773636817932
1.2358412742614746
0.43190717697143555
1.2455670833587646
0.40694403648376465
1.202174425125122
0.38221096992492676
1.3409132957458496
0.3565516769886017
1.1871023178100586
0.3280313014984131
1.1472444534301758
0.29837411642074585
1.2010293006896973
0.2676803767681122
1.1323425769805908
0.23611629009246826
1.1967949867248535
0.2039056122303009
1.1556222438812256
0.17220370471477509
1.1893057823181152
0.14112982153892517
1.216019630432129
0.11364461481571198
1.1702077388763428
0.08855108171701431
1.215672492980957
0.06682920455932617
1.183851957321167
0.0493365153670311
1.1578898429870605
0.03423932194709778
1.2125740051269531
0.024068936705589294
1.1692237854003906


In [177]:
#y=p([i for i in range(0,512)]).T


tensor([[ 0.0205,  0.0498, -0.0723,  ..., -0.0156,  0.0071, -0.0528],
        [ 0.0084,  0.0193, -0.0749,  ..., -0.0320,  0.0058, -0.0857],
        [ 0.0276,  0.0566, -0.0652,  ..., -0.0214,  0.0234, -0.0616],
        ...,
        [ 0.0254,  0.0401, -0.0572,  ..., -0.0179,  0.0199, -0.0590],
        [ 0.0108,  0.0490, -0.0623,  ..., -0.0176,  0.0159, -0.0556],
        [ 0.0116,  0.0430, -0.0654,  ..., -0.0282,  0.0179, -0.0665]],
       grad_fn=<PermuteBackward0>)

In [161]:
d=p.tracing([i for i in range(512)])
Es=torch.randn(256,7646)
p.select(Es,set(),d[1])

RuntimeError: INDICES element is out of DATA bounds, id=7646 axis_dim=7646

7646

{0: [[11146], [6528]],
 1: [[], []],
 2: [[], [1235]],
 3: [[603,
   1147,
   1273,
   2202,
   2209,
   3095,
   5104,
   5876,
   5894,
   6349,
   7210,
   7692,
   8334,
   8670,
   8870,
   9690,
   10576,
   10744],
  [5755,
   1200,
   8359,
   8436,
   4095,
   7133,
   572,
   1137,
   1799,
   3305,
   6109,
   2539,
   1869,
   1582,
   4768,
   6123,
   673,
   8819]],
 4: [[9245, 9664, 10102, 10510], [5237, 7522, 3870, 5553]],
 6: [[3687, 4170, 5223, 10546, 11033, 11174],
  [2188, 3325, 7011, 6658, 116, 7032]],
 7: [[12,
   337,
   1140,
   1341,
   1887,
   3223,
   4253,
   4893,
   7181,
   8118,
   8396,
   9051,
   9852],
  [1004,
   1527,
   7238,
   1479,
   5837,
   7538,
   6388,
   8106,
   4374,
   2134,
   3538,
   2962,
   8810]],
 8: [[6637, 7074, 9501, 10253, 10523], [4396, 1148, 7806, 6203, 2695]],
 9: [[107, 1417, 1518, 5636, 8502, 10218],
  [7600, 8717, 8096, 1164, 6475, 4119]],
 10: [[1784, 1872, 2433, 2598, 3594, 5196, 6101, 8503, 8694, 9436],
  [5574, 

In [68]:
dd=dz.copy()
for i in ins:
    if i in dd[2][0]:
        dd[2][0].pop(ins.index(i)) 
        print(i)
        dd[i][0].pop(ins.index(2))
    elif 2 in dd[2][1]:
        dd[2][1].pop(ins.index(i)) 
        dd[i][1].pop(ins.index(2))
dd   

{0: [[11146], [6528]],
 1: [[], []],
 2: [[], []],
 3: [[603,
   1147,
   1273,
   2202,
   2209,
   3095,
   5104,
   5876,
   5894,
   6349,
   7210,
   7692,
   8334,
   8670,
   8870,
   9690,
   10576,
   10744],
  [5755,
   1200,
   8359,
   8436,
   4095,
   7133,
   572,
   1137,
   1799,
   3305,
   6109,
   2539,
   1869,
   1582,
   4768,
   6123,
   673,
   8819]],
 4: [[9245, 9664, 10102, 10510], [5237, 7522, 3870, 5553]],
 6: [[3687, 4170, 5223, 10546, 11033, 11174],
  [2188, 3325, 7011, 6658, 116, 7032]],
 7: [[12,
   337,
   1140,
   1341,
   1887,
   3223,
   4253,
   4893,
   7181,
   8118,
   8396,
   9051,
   9852],
  [1004,
   1527,
   7238,
   1479,
   5837,
   7538,
   6388,
   8106,
   4374,
   2134,
   3538,
   2962,
   8810]],
 8: [[6637, 7074, 9501, 10253, 10523], [4396, 1148, 7806, 6203, 2695]],
 9: [[107, 1417, 1518, 5636, 8502, 10218],
  [7600, 8717, 8096, 1164, 6475, 4119]],
 10: [[1784, 1872, 2433, 2598, 3594, 5196, 6101, 8503, 8694, 9436],
  [5574, 519,

In [34]:
dz=gs.diz
bs=512


8729

{0: [[1386, 5175, 6682, 8154, 8968, 11146],
  [7093, 2848, 7872, 4270, 181, 7300]],
 1: [[6739], [5975]],
 2: [[9198, 9332, 10688, 11030], [5290, 1393, 2271, 1677]],
 3: [[603,
   1147,
   1273,
   2202,
   2209,
   3095,
   5104,
   5876,
   5894,
   6349,
   7210,
   7692,
   8334,
   8670,
   8870,
   9690,
   10576,
   10744],
  [4147,
   7081,
   2714,
   1839,
   2452,
   7390,
   6198,
   6901,
   5642,
   1101,
   8325,
   6271,
   2387,
   7590,
   4733,
   7576,
   7606,
   136]],
 4: [[248,
   336,
   861,
   2231,
   4057,
   4496,
   4979,
   5009,
   5443,
   5676,
   5820,
   7355,
   7749,
   8145,
   8857,
   9245,
   9664,
   10102,
   10510],
  [6341,
   8162,
   5167,
   4992,
   4773,
   3461,
   452,
   8581,
   1826,
   4525,
   8028,
   3316,
   5955,
   2761,
   2548,
   5496,
   2756,
   1128,
   6963]],
 6: [[3687, 4170, 5223, 10546, 11033, 11174],
  [6139, 2991, 5519, 8788, 7315, 3570]],
 7: [[12,
   337,
   1140,
   1341,
   1887,
   3223,
   4253,
   4893,

In [37]:
dmb=DMB(dz.copy(),training,bs)
dmb.getPN(3,set())

P is: 3
Sp is: set()
len of dict is: 8729
Element in diz for P are [[603, 1273, 2202, 3095, 5104, 5876, 6349, 7210, 7692, 8334, 8870, 9690, 10576, 10744], [237, 3453, 5124, 965, 5223, 54, 4376, 2119, 7404, 25, 7955, 4506, 7162, 4729]]
lenSp 8
lenSp1 8
lenSn1 4
P is: 1273
Sp is: {2119, 7210, 6349, 237, 4376, 1273, 603, 3453}
len of dict is: 8729
Element in diz for P are [[83, 692, 1624, 2018, 2633, 2801, 3036, 4466, 4929, 5401, 6258, 6631, 6746, 6916, 6950, 7707, 8440, 9285, 9552, 9608, 9660, 11236], [8045, 4512, 5756, 941, 8006, 4344, 6418, 8560, 6348, 7635, 4536, 2359, 1328, 149, 5067, 4441, 8562, 8016, 4243, 1150, 7086, 2329, 881]]
lenSp 16
lenSp1 8
lenSn1 4
P is: 8440
Sp is: {8440, 4929, 6950, 2119, 7210, 5067, 6348, 6349, 237, 1328, 8562, 4376, 1273, 6746, 603, 3453}
len of dict is: 8729
Element in diz for P are [[692, 1624, 1919, 2037, 2178, 2396, 2801, 4929, 6631, 6746, 6965, 7153, 7716, 9285, 9552, 9593, 10666, 11236], [4239, 3340, 6367, 1243, 4763, 3186, 4927, 3875, 4631, 6996,

KeyboardInterrupt: 

[9198, 9332, 10688, 11030]

In [36]:
class DMB:
    def __init__(self,diz,training,bs):
        self.diz=diz
        self.training=training
        self.bs=bs
        self.M=4
    def getMB(self):
        mbS=[]
        while len(self.diz)!=0:
            mb=set()
            c=0
            while c<len(self.diz) and len(mb)<self.bs:
                f=self.getPN(c,mb,self.M)
                print(f)
                mb=mb.union(f)
                print(mb)
                if len(mb)>self.bs:
                    mbS.append(mb)
                    break
                else:
                    c+=1
        return mbS
    def getPN(self,P,Sp,M=4):
        print("P is:",P)
        print("Sp is:",Sp)
        print("len of dict is:",len(self.diz))
        print("Element in diz for P are",self.diz[P])
        if len(Sp)==self.bs:
            return Sp

        Sp1=set()
        Sn1=set()
        while len(self.diz[P][0])>0: #self.diz
            if len(set(self.diz[P][0]).intersection(set(self.training)))==0 or len(set(self.diz[P][1]).intersection(set(self.training)))==0: #self.diz, self.training
                self.diz.pop(P)
                return set([P])
            i=randrange(1000)
            random.seed(i)
            r=randrange(len(self.diz[P][0])) #self.diz
            while self.diz[P][0][r] not in self.training: #self.training
                i=randrange(1000)
                random.seed(i)
                r=randrange(len(self.diz[P][0]))#self.diz 
            p=self.diz[P][0][r]              #self.diz

            while self.diz[P][1][r] not in self.training: #self.training
                i=randrange(1000)
                random.seed(i)
                r=randrange(len(self.diz[P][0])) #self.diz 
                
            n=self.diz[P][1][r]  #self.diz
            Sp1.add(p)
            Sn1.add(n)
            if len(Sp1)==M:
                break
        
        
        
        if len(Sp)+ len(Sp1) + len(Sn1) >self.bs:
            diff= len(Sp)- (len(Sp1)+len(Sn1)+1)
            Sp.add([P])
            l=list(Sp1.union(Sn1))
            S=set(l[0:diff])
            for i in S:
                if i in self.diz[P][0]:
                    self.diz[P][0].pop(self.diz[P][0].index(i)) 
                    self.diz[i][0].pop(self.diz[i][0].index(P))
                elif i in self.diz[P][1]:
                    self.diz[P][1].pop(self.diz[P][1].index(i)) 
            Sp=Sp.union(S)
            return Sp
        
        elif len(Sp)+len(Sp1)+len(Sn1)==self.bs:
            Sp.add([P])
            S=set(list(Sp1.union(Sn1))[-1])
            for i in S:
                if i in self.diz[P][0]:
                    self.diz[P][0].pop(self.diz[P][0].index(i)) 
                    self.diz[i][0].pop(self.diz[i][0].index(P))
                elif i in self.diz[P][1]:
                    self.diz[P][1].pop(self.diz[P][1].index(i)) 
            Sp=Sp.union(S)
            return Sp
        else:
            S=Sp1.union(Sn1)
            for i in S:
                if i in self.diz[P][0]:
                    self.diz[P][0].pop(self.diz[P][0].index(i)) 
                    self.diz[i][0].pop(self.diz[i][0].index(P))
                elif i in self.diz[P][1]:
                    self.diz[P][1].pop(self.diz[P][1].index(i)) 
            for k in Sp1:
                Sp1=Sp1.union(Sn1)
                Sp=Sp.union(Sp1)
                print("lenSp",len(Sp))
                print("lenSp1",len(Sp1))
                print("lenSn1",len(Sn1))
                Sp=Sp.union(self.getPN(k,Sp))
                if len(Sp)>self.bs:
                    print("out of space")
                    break
                elif len(Sp)==self.bs:
                    break
        
        return Sp
        
                    

In [2]:
Sp=set([3,4,5,6,7])

In [18]:
g=set([45,6,2,4])
Sp.add(28)

In [19]:
Sp

{3, 4, 5, 6, 7, 28}

## First Configuration #1

- Through torch_geometric we have to use the Coordinate format (COO), to represent the graph data structure.
- Thus, in the class definition for the first configuration requested we have to convert our Adjacency matrix in a COO format.

- Network design requested:

1. Linear(Input, 256) 
2. Linear(256, 256)
3. GCNConv(256, 256)
4. GCNConv(256, 256)
5. TripletLoss()

In [18]:
n_input=2613
train_=list(range(0,9021+1)) #Train set, without val
train=list(range(0,10189+1)) #Train set, with val
val=list(range(9022,10189+1))
test=list(range(10190,11260+1))
KNN=200       #K-nearest-neighbors for the evaluation metrics.
class Conf1(nn.Module):
    
    def __init__(self,X,A):
        super(Conf1,self).__init__()
        self.X=X
        self.A=A
        self.l1=nn.Linear(n_input,256)
        self.l2=nn.Linear(256,256)
        self.GCN1=GCNConv(256,256)
        self.GCN2=GCNConv(256,256)
        
        self.COO=torch.load('COOA.pt')
        
    
    def forward(self,V):
        Xnew=torch.transpose(self.select(self.X,set(),V),0,1)
        Anew=self.select(self.A,V,V)
        Anew=self.ConvertAtoCOO(Anew).type(torch.LongTensor)
        Xnew=torch.tensor(Xnew.tolist(),requires_grad=True)
        Xnew=self.l1(Xnew)
        Xnew=F.elu(Xnew)
        Xnew=self.l2(Xnew)
        Xnew=F.elu(Xnew)
        Xnew=self.GCN1(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.GCN2(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=torch.transpose(Xnew,0,1)
        return Xnew
    
    
    
    
    def select(self,mat,row,col):  #Given a set of indices for rows or column or both, we get the respective elements.
        col=sorted(list(col))      #This is applied when we get the t matrix.
        
        c=0
        if row==set():
            ma=torch.zeros((mat.shape[0],len(col)))
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return ma
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def ConvertAtoCOO(self,SA):
        Anew=torch.tensor([[],[]])
        summ=SA.sum().item()
        for i in range(SA.shape[0]):
            for j in range(SA.shape[1]):
                if SA[i][j]!=0:
                    Anew=torch.cat((Anew,torch.tensor([[i],[j]])),dim=1)
        if summ!=Anew.shape[1]:
            print("error in conversion....")
        return Anew
    
    def mini_batches(self,indices,bs=32): #This function generates a list of minibatches, of size bs
        indicesN=indices.copy()           #sets are unordered data structure, so there is no need to shuffle them. 
        mbList=[]                         #Lists of lists of mini_batches indices 
        while len(indicesN)!=0:
            mb=set()                      #Inner list, with the indices of a particular mini_batch
            while len(mb)<bs:
                if len(indicesN)==0:
                    mbList.append(mb)
                    return mbList
                r=choice(indicesN)
                sample=indicesN.pop(indicesN.index(r))
                mb.add(sample)
            mbList.append(mb)
        return mbList          #obj.mini_batches(#,bs=128) #: train_,train,val,test, we get lists of list of batches from here
    def calcG(self,ID):  #This method is used for the evaluation of accuracy, in particular it computes the denominator
        if ID>200:       # as described in the paper.
            ID=200
        c=1
        somm=0
        while c<=ID:
            somm+=1/(math.log2(1+c))
            c+=1
        return somm

    def evalAcc(self,T,S,kneigh):  #This function is to compute accuracy for the test and train set. 
        T=T.detach().numpy().transpose()
        neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)  #With the K-NN we get the nearest 
        dist,ind=neigh.kneighbors(T)                                                 #neighbors in the embedded.
        acc=[]                          
        for k in S:
            summ=0
            ideal=self.A[k,:].sum().item()  #gs
            den=self.calcG(ideal)
            c=1
            if den==0:#There is the problem of the distance for the people without neighbors
                continue  #1-(n/200) or ignore them.
            for j in ind[k][1:]:
                if self.A[k][j]!=0:
                    summ+= 1/(math.log2(1+c))
                else:
                    continue
                c+=1
            summ/=den
            acc.append(summ)
        return acc
    
    def tfunc(self,n_feat,es,V):               #This is the t function, which was previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    def tfunc2(self,n_feat,es,V,prev):               #This tfunction is later used for the accuracy evaluation step
        V=sorted(list(V))
        c=0
        for k in V:
            prev[:,k]=es[:,c]
            c+=1
        return prev
    

In [12]:
d

NameError: name 'd' is not defined

In [ ]:
d={1:[3,4,5,6],2:[3,21,2,3]}

## Training step

## Second Configuration #2

- Through torch_geometric we have to use the Coordinate format (COO), to represent the graph data structure.
- Thus, in the class definition for the first configuration requested we have to convert our Adjacency matrix in a COO format.

- Network design requested:

1. GCNConv(Input, 256)
2. GraphConv(256, 256)
3. GCNConv(256, 256)
4. GCNConv(256, 256)
5. TripletLoss()

In [29]:
n_input=2613
train_=list(range(0,9021+1)) #Train set, without val
train=list(range(0,10189+1)) #Train set, with val
val=list(range(9022,10189+1))
test=list(range(10190,11260+1))
KNN=200       #K-nearest-neighbors for the evaluation metrics.
class Conf2(nn.Module):
    
    def __init__(self,X,A):
        super(Conf2,self).__init__()
        self.X=X
        self.A=A
        self.GCN1=GCNConv(n_input,256)
        self.Graph=GraphConv(256,256)
        self.GCN2=GCNConv(256,256)
        self.GCN3=GCNConv(256,256)
        
    
    
    def forward(self,V):
        Xnew=torch.transpose(self.select(self.X,set(),V),0,1)
        Xnew=torch.tensor(Xnew.tolist(),requires_grad=True)
        Anew=self.select(self.A,V,V)
        Anew=self.ConvertAtoCOO(Anew).type(torch.LongTensor)
        Xnew=self.GCN1(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.Graph(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.GCN2(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.GCN3(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=torch.transpose(Xnew,0,1)
        return Xnew
    
    
    
    
    def select(self,mat,row,col):  #Given a set of indices for rows or column or both, we get the respective elements.
        col=sorted(list(col))      #This is applied when we get the t matrix.
        
        c=0
        if row==set():
            ma=torch.zeros((mat.shape[0],len(col)))
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return ma
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def ConvertAtoCOO(self,SA):
        Anew=torch.tensor([[],[]])
        summ=SA.sum().item()
        for i in range(SA.shape[0]):
            for j in range(SA.shape[1]):
                if SA[i][j]!=0:
                    Anew=torch.cat((Anew,torch.tensor([[i],[j]])),dim=1)
        if summ!=Anew.shape[1]:
            print("error in conversion....")
        return Anew
    
    def mini_batches(self,indices,bs=32): #This function generates a list of minibatches, of size bs
        indicesN=indices.copy()           #sets are unordered data structure, so there is no need to shuffle them. 
        mbList=[]                         #Lists of lists of mini_batches indices 
        while len(indicesN)!=0:
            mb=set()                      #Inner list, with the indices of a particular mini_batch
            while len(mb)<bs:
                if len(indicesN)==0:
                    mbList.append(mb)
                    return mbList
                r=choice(indicesN)
                sample=indicesN.pop(indicesN.index(r))
                mb.add(sample)
            mbList.append(mb)
        return mbList          #obj.mini_batches(#,bs=128) #: train_,train,val,test, we get lists of list of batches from here
    def calcG(self,ID):  #This method is used for the evaluation of accuracy, in particular it computes the denominator
        if ID>200:       # as described in the paper.
            ID=200
        c=1
        somm=0
        while c<=ID:
            somm+=1/(math.log2(1+c))
            c+=1
        return somm

    def evalAcc(self,T,S,kneigh):  #This function is to compute accuracy for the test and train set. 
        T=T.detach().numpy().transpose()
        neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)  #With the K-NN we get the nearest 
        dist,ind=neigh.kneighbors(T)                                                 #neighbors in the embedded.
        acc=[]                          
        for k in S:
            summ=0
            ideal=self.A[k,:].sum().item()  #gs
            den=self.calcG(ideal)
            c=1
            if den==0:#There is the problem of the distance for the people without neighbors
                continue  #1-(n/200) or ignore them.
            for j in ind[k][1:]:
                if self.A[k][j]!=0:
                    summ+= 1/(math.log2(1+c))
                else:
                    continue
                c+=1
            summ/=den
            acc.append(summ)
        return acc
    
    def tfunc(self,n_feat,es,V):               #This is the t function, which was previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    def tfunc2(self,n_feat,es,V,prev):               #This tfunction is later used for the accuracy evaluation step
        V=sorted(list(V))
        c=0
        for k in V:
            prev[:,k]=es[:,c]
            c+=1
        return prev
       
    

## Third Configuration #3

- Through torch_geometric we have to use the Coordinate format (COO), to represent the graph data structure.
- Thus, in the class definition for the first configuration requested we have to convert our Adjacency matrix in a COO format.

- Network design requested:

1. Linear(Input, 256)
2. Linear(256, 256)
3. GATConv(256, 256)
4. GATConv(256, 256)
5. TripletLoss()

In [9]:
n_input=2613
train_=list(range(0,9021+1)) #Train set, without val
train=list(range(0,10189+1)) #Train set, with val
val=list(range(9022,10189+1))
test=list(range(10190,11260+1))
KNN=200       #K-nearest-neighbors for the evaluation metrics.
class Conf3(nn.Module):
    
    def __init__(self,X,A):
        super(Conf3,self).__init__()
        self.X=X
        self.A=A
        self.l1=nn.Linear(n_input,256)
        self.l2=nn.Linear(256,256)
        self.GAT1=GATConv(256,256)
        self.GAT2=GATConv(256,256)
        
    
    
    def forward(self,V):
        Xnew=torch.transpose(self.select(self.X,set(),V),0,1)
        Xnew=torch.tensor(Xnew.tolist(),requires_grad=True)
        Anew=self.select(self.A,V,V)
        Anew=self.ConvertAtoCOO(Anew).type(torch.LongTensor)
        Xnew=self.l1(Xnew)
        Xnew=F.elu(Xnew)
        Xnew=self.l2(Xnew)
        Xnew=F.elu(Xnew)
        Xnew=self.GAT1(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.GAT2(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=torch.transpose(Xnew,0,1)
        return Xnew
    
    
    
    
    def select(self,mat,row,col):  #Given a set of indices for rows or column or both, we get the respective elements.
        col=sorted(list(col))      #This is applied when we get the t matrix.
        
        c=0
        if row==set():
            ma=torch.zeros((mat.shape[0],len(col)))
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return ma
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def ConvertAtoCOO(self,SA):
        Anew=torch.tensor([[],[]])
        summ=SA.sum().item()
        for i in range(SA.shape[0]):
            for j in range(SA.shape[1]):
                if SA[i][j]!=0:
                    Anew=torch.cat((Anew,torch.tensor([[i],[j]])),dim=1)
        if summ!=Anew.shape[1]:
            print("error in conversion....")
        return Anew
    
    def mini_batches(self,indices,bs=32): #This function generates a list of minibatches, of size bs
        indicesN=indices.copy()           #sets are unordered data structure, so there is no need to shuffle them. 
        mbList=[]                         #Lists of lists of mini_batches indices 
        while len(indicesN)!=0:
            mb=set()                      #Inner list, with the indices of a particular mini_batch
            while len(mb)<bs:
                if len(indicesN)==0:
                    mbList.append(mb)
                    return mbList
                r=choice(indicesN)
                sample=indicesN.pop(indicesN.index(r))
                mb.add(sample)
            mbList.append(mb)
        return mbList          #obj.mini_batches(#,bs=128) #: train_,train,val,test, we get lists of list of batches from here
    def calcG(self,ID):  #This method is used for the evaluation of accuracy, in particular it computes the denominator
        if ID>200:       # as described in the paper.
            ID=200
        c=1
        somm=0
        while c<=ID:
            somm+=1/(math.log2(1+c))
            c+=1
        return somm

    def evalAcc(self,T,S,kneigh):  #This function is to compute accuracy for the test and train set. 
        T=T.detach().numpy().transpose()
        neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)  #With the K-NN we get the nearest 
        dist,ind=neigh.kneighbors(T)                                                 #neighbors in the embedded.
        acc=[]                          
        for k in S:
            summ=0
            ideal=self.A[k,:].sum().item()  #gs
            den=self.calcG(ideal)
            c=1
            if den==0:#There is the problem of the distance for the people without neighbors
                continue  #1-(n/200) or ignore them.
            for j in ind[k][1:]:
                if self.A[k][j]!=0:
                    summ+= 1/(math.log2(1+c))
                else:
                    continue
                c+=1
            summ/=den
            acc.append(summ)
        return acc
    
    def tfunc(self,n_feat,es,V):               #This is the t function, which was previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    def tfunc2(self,n_feat,es,V,prev):               #This tfunction is later used for the accuracy evaluation step
        V=sorted(list(V))
        c=0
        for k in V:
            prev[:,k]=es[:,c]
            c+=1
        return prev
            
      
    

## Fourth Configuration #4

- Through torch_geometric we have to use the Coordinate format (COO), to represent the graph data structure.
- Thus, in the class definition for the first configuration requested we have to convert our Adjacency matrix in a COO format.

- Network design requested:

1. GATConv(Input, 256)
2. GATConv(256, 256)
3. Linear(256, 256)
4. Linear(256, 256)
5. TripletLoss()

In [16]:
n_input=2613
train_=list(range(0,9021+1)) #Train set, without val
train=list(range(0,10189+1)) #Train set, with val
val=list(range(9022,10189+1))
test=list(range(10190,11260+1))
KNN=200       #K-nearest-neighbors for the evaluation metrics.
class Conf4(nn.Module):
    
    def __init__(self,X,A):
        super(Conf4,self).__init__()
        self.X=X
        self.A=A
        self.l1=nn.Linear(256,256)
        self.l2=nn.Linear(256,256)
        self.GAT1=GATConv(n_input,256)
        self.GAT2=GATConv(256,256)
        
    
    
    def forward(self,V):
        Xnew=torch.transpose(self.select(self.X,set(),V),0,1)
        Anew=self.select(self.A,V,V)
        Anew=self.ConvertAtoCOO(Anew).type(torch.LongTensor)
        Xnew=self.GAT1(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.GAT2(Xnew,Anew)
        Xnew=F.elu(Xnew)
        Xnew=self.l1(Xnew)
        Xnew=F.elu(Xnew)
        Xnew=self.l2(Xnew)
        Xnew=F.elu(Xnew)
        Xnew=torch.transpose(Xnew,0,1)
        return Xnew
    
    
    
    
    def select(self,mat,row,col):  #Given a set of indices for rows or column or both, we get the respective elements.
        col=sorted(list(col))      #This is applied when we get the t matrix.
        
        c=0
        if row==set():
            ma=torch.zeros((mat.shape[0],len(col)))
            for k in col:
                ma[:,c]=mat[:,k]
                c+=1
            return ma
        else:
            row=torch.tensor(sorted(list(row)))
            col=torch.tensor(col)
            ma=torch.index_select(mat,0,row)
            ma=torch.index_select(ma,1,col)
            return ma
    def ConvertAtoCOO(self,SA):
        Anew=torch.tensor([[],[]])
        summ=SA.sum().item()
        for i in range(SA.shape[0]):
            for j in range(SA.shape[1]):
                if SA[i][j]!=0:
                    Anew=torch.cat((Anew,torch.tensor([[i],[j]])),dim=1)
        if summ!=Anew.shape[1]:
            print("error in conversion....")
        return Anew
    
    def mini_batches(self,indices,bs=32): #This function generates a list of minibatches, of size bs
        indicesN=indices.copy()           #sets are unordered data structure, so there is no need to shuffle them. 
        mbList=[]                         #Lists of lists of mini_batches indices 
        while len(indicesN)!=0:
            mb=set()                      #Inner list, with the indices of a particular mini_batch
            while len(mb)<bs:
                if len(indicesN)==0:
                    mbList.append(mb)
                    return mbList
                r=choice(indicesN)
                sample=indicesN.pop(indicesN.index(r))
                mb.add(sample)
            mbList.append(mb)
        return mbList          #obj.mini_batches(#,bs=128) #: train_,train,val,test, we get lists of list of batches from here
    def calcG(self,ID):  #This method is used for the evaluation of accuracy, in particular it computes the denominator
        if ID>200:       # as described in the paper.
            ID=200
        c=1
        somm=0
        while c<=ID:
            somm+=1/(math.log2(1+c))
            c+=1
        return somm

    def evalAcc(self,T,S,kneigh):  #This function is to compute accuracy for the test and train set. 
        T=T.detach().numpy().transpose()
        neigh=NearestNeighbors(n_neighbors=(kneigh+1),algorithm='ball_tree').fit(T)  #With the K-NN we get the nearest 
        dist,ind=neigh.kneighbors(T)                                                 #neighbors in the embedded.
        acc=[]                          
        for k in S:
            summ=0
            ideal=self.A[k,:].sum().item()  #gs
            den=self.calcG(ideal)
            c=1
            if den==0:#There is the problem of the distance for the people without neighbors
                continue  #1-(n/200) or ignore them.
            for j in ind[k][1:]:
                if self.A[k][j]!=0:
                    summ+= 1/(math.log2(1+c))
                else:
                    continue
                c+=1
            summ/=den
            acc.append(summ)
        return acc
    
    def tfunc(self,n_feat,es,V):               #This is the t function, which was previously described
        t=torch.zeros((n_feat,self.X.shape[1]))
        V=sorted(list(V))
        c=0
        for k in V:
            t[:,k]=es[:,c]
            c+=1
        return t
    def tfunc2(self,n_feat,es,V,prev):               #This tfunction is later used for the accuracy evaluation step
        V=sorted(list(V))
        c=0
        for k in V:
            prev[:,k]=es[:,c]
            c+=1
        return prev
            
      
    

# Training step

In [16]:
conf=1

if conf==1:
    c=Conf1(X1,A1)
elif conf==2:
    c=Conf2(X1,A1)
elif conf==3:
    c=Conf3(X1,A1)
elif conf==4:
    c=Conf4(X1,A1)

training=train  # train_, train
if training==train_:
    f="trainval"
elif training==train:
    f="train"
testing=test  #val, test
batch_size=512 
mbb=c.mini_batches(training,bs=batch_size)
num_epochs=1 #According to the paper there will be 50 epochs for each experiment 
out_feat=256
triplet_loss = nn.TripletMarginLoss(margin=0.2, p=2)
optimizer=torch.optim.Adam(c.parameters(),lr=0.001)

In [17]:
#With these lines of code we obtain the embedded space sample
start=time.time()
#This path has to be modified.
print("Training of the {}-th net configuration, with the {} partition".format(conf,f))
path="C:\\Users\\Peppe\\OneDrive\\Desktop\\Università\\magistrale\\Neural_Networks\\MIR_project\\minibatches\\conf" + str(conf) + "\\"+ str(f)+"_batches\\file"
for epoch in range(num_epochs):
    print("Processing epoch n° ",epoch+1)
    num=int(len(training)/batch_size)+1     
    for k in range(len(mbb)):
        if k==1:
            break
        Ex=c(mbb[k])
        #TODO: Loss function, Optimizer
        name=path+str(k)+".pt"
        torch.save(Ex,name)
    
    for k in range(len(mbb)):
        name=path+str(k)+".pt"
        ex=torch.load(name)
        
        if k==0:
            t1=c.tfunc(out_feat,ex,mbb[k])

        else:
            t1=c.tfunc2(out_feat,ex,mbb[k],t1)
    break
    print("Evaluating the epoch n° ",epoch+1)
    accL1=c.evalAcc(t1[:,:training[-1]+1],set(training),KNN)
    t2=c(set(testing))
    t2=c.tfunc2(outf[-1],t2,set(testing),t1) #We integrate the testing set to the previous training set
    accL2=c.evalAcc(t2,set(testing),KNN)
    TestAcc=sum(accL1)/len(accL1)
    TrainAcc=sum(accL2)/len(accL2)
    print("Processesed epoch n° {}, \tTrain accuracy: {:.4f}, \tTest accuracy: {:.4f}".format((epoch+1),TrainAcc,TestAcc))
    print("done")
end=time.time()
print(end-start) 

Training of the 1-th net configuration, with the train partition
Processing epoch n°  1
2.893789529800415


In [27]:
anchors=torch.randn(512,256,requires_grad=True)
positives=torch.randn(512,256,requires_grad=True)
negatives=torch.randn(512,256,requires_grad=True)
tl=triplet_loss(Exnew,positives,negatives)    
optimizer.zero_grad()
tl.backward()
optimizer.step()


In [26]:
Exnew=torch.transpose(Ex,0,1).tolist()
Exnew=torch.tensor(Exnew,requires_grad=True)


tensor([[ 0.2361,  0.0504, -0.1597,  ...,  0.1840, -0.0293,  0.1570],
        [-0.4286,  0.3652, -0.0188,  ..., -0.1241,  0.4381,  0.1229],
        [-0.2130, -0.1340, -0.0267,  ..., -0.1231,  0.0229,  0.0788],
        ...,
        [-0.0843, -0.3757, -0.1636,  ..., -0.0185, -0.1687, -0.0337],
        [-0.0625,  0.1115, -0.0804,  ...,  0.0591, -0.0250,  0.1086],
        [-0.1814, -0.1057, -0.1372,  ...,  0.0546,  0.1489, -0.0676]],
       requires_grad=True)